Step 1: Data Cleaning and Saving Cleaned Versions

In [40]:
import pandas as pd
import numpy as np

creditcard_df = pd.read_csv('../data/creditcard.csv')
fraud_data_df = pd.read_csv('../data/Fraud_Data.csv')

In [41]:
creditcard_df.drop_duplicates(inplace=True)

In [42]:
creditcard_df['Time'] = pd.to_datetime(creditcard_df['Time'], unit='s', origin='2010-01-01')

In [43]:
creditcard_df.to_csv('../data/cleaned_creditcard.csv', index=False)

In [44]:
fraud_data_df.drop_duplicates(inplace=True)

In [45]:
fraud_data_df['signup_time'] = pd.to_datetime(fraud_data_df['signup_time'])
fraud_data_df['purchase_time'] = pd.to_datetime(fraud_data_df['purchase_time'])